In [1]:
import pandas as pd
import numpy as np
import altair as alt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics

In [2]:
# Import dataframe column names and helper functions
%run -i columns.py
%run -i helper_functions.py

In [3]:
"""
Creates a train set and a test set from the given given dataframe instance. The name and code of the country, its
Dystopia residual value, and its Dystopia residual-to-happiness score ratio are always present in the resulting datasets. 
Any other variable to be included in the dataset must be named in the given list of indicators.

Parameters:
    df (pd.DataFrame): The dataframe of developing countries to process and split into train and test sets
    indicators (list): The list of positive or negative indicators of urbanization to include in the resulting datasets

Returns: two dataframes, a train set which contains 80% of the original dataframe's entries, and a test set containing
         the remaining entries
"""
def create_train_test_sets(df: pd.DataFrame, indicators: list) -> (pd.DataFrame, pd.DataFrame):
    columns = ['Country', 'Country code', 'Dystopia residual', 'Residual-to-happiness ratio'] + indicators
    df = df[columns] 
    
    # Discard any indicators for which more than 50% of the dataframe's entries are missing data
    threshold = 0.5
    df_thresh = df.dropna(axis=1, thresh=int(df.shape[0] * threshold))
    
    # Impute the remaining NaN values using the mean imputation method
    df_impute = df_thresh.fillna(df_thresh.mean())
    
    train, test = train_test_split(df_impute, test_size=0.2)
    
    return train, test

In [4]:
"""
A multiple linear regression model.
"""
class LinearRegressionModel():
    """
    Creates a new model using the given dataframe, in which the independent variables are in a list of variable names, x, and
    the name of the dependent variable is a string, y.
    """
    def __init__(self, df: pd.DataFrame, x: list, y: str):
        # Prepare the train and test datasets to be used by the model
        self.train, self.test = create_train_test_sets(df.copy(), x)
        self.df = pd.concat([self.train.copy(), self.test.copy()])
        
        self.x = list(set(x).intersection(self.train.columns))
        self.y = y
        
        # Train the model using the train set, then calculate the predicted outputs using the test set
        self.model = LinearRegression()
        self.model.fit(self.train[self.x], self.train[self.y])
        self.predictions = self.model.predict(self.test[self.x])
        
    """
    Returns a copy of the test dataframe which contains the predicted and actual values of the dependent variable
    for all entries.
    """
    def get_predictions_table(self) -> pd.DataFrame:
        table = pd.DataFrame(self.test[self.x].copy())
        table[f'Predicted {self.y}'] = list(self.predictions)
        table[f'Actual {self.y}'] = list(self.test[self.y])
        
        return table
    
    """
    Returns a dataframe that contains the coefficients of all independent variables.
    """
    def get_coefficients_table(self) -> pd.DataFrame:
        coefficients = pd.DataFrame(self.train[self.x].columns, columns=['variable'])
        coefficients['coefficient'] = self.model.coef_

        return coefficients
    
    """
    Returns the y-intercept of the model.
    """
    def get_intercept(self) -> float:
        return self.model.intercept_
    
    """
    Returns the R^2 value of the model.
    """
    def get_r_squared(self) -> float:
        return self.model.score(self.train[self.x], self.train[self.y])
    
    """
    Returns the mean squared error of the model's predictinos.
    """
    def get_mean_squared_error(self) -> float:
        return metrics.mean_squared_error(self.test[self.y], self.predictions)
    
    """
    Displays the R^2 value, MSE, y-intercept, and coefficients of the model.
    """
    def display_properties(self) -> pd.DataFrame:
        print("R\u00B2:", self.get_r_squared())
        print("MSE:", self.get_mean_squared_error())

        print("Y-intercept:", self.get_intercept())
        print("Coefficients:")
        return self.get_coefficients_table()
    
    """
    Performs PCA on the train dataset, and returns a scree plot.
    """
    def get_scree_plot(self) -> alt.Chart:
        # Set K to be the number of independent variables
        k = len(self.train[self.x].columns)
        
        # Scale the independent variables, and perform PCA using them
        scaled_values = StandardScaler().fit(self.train[self.x]).transform(self.train[self.x])
        pca = PCA(n_components=k).fit(scaled_values)
        
        # Produce a dataframe for the scree plot
        scree_plot_data = pd.DataFrame()
        scree_plot_data['indices'] = np.arange(k) + 1
        scree_plot_data['var_explained'] = pca.explained_variance_ratio_
        
        return alt.Chart(scree_plot_data, title='Scree plot').mark_line().encode(
            x=alt.X('indices', title='Principal component'),
            y=alt.Y('var_explained', title='Variance explained')
        )
    
    """
    Creates a scree plot and a 2D visualization of the data using PCA with a K value of 1.
    """
    def visualize(self) -> alt.Chart:
        # Scale the inputs of the dataframe, and perform PCA with K = 1
        scaled_inputs = StandardScaler().fit(self.df[self.x]).transform(self.df[self.x])
        pca = PCA(n_components=1).fit(scaled_inputs).transform(scaled_inputs)
        
        # Create a dataframe for the PCA results
        results = pd.DataFrame(pca, columns=['PC0'])
        results[self.y] = self.df[self.y]
        results['Country'] = self.df['Country code']
        
        # Produce a scatter plot of the principal component vs. the Dystopia residual-to-happiness ratio
        chart = alt.Chart(results, title=f'PC0 vs {self.y}').mark_point().encode(
            x='PC0',
            y=self.y,
            color='Country:N',
            tooltip=['Country', 'PC0', self.y]
        )
        
        # Produce a linear regression for the scatter plot
        # Based on https://stackoverflow.com/questions/66604052/altair-extract-and-display-regression-coefficients
        regression = chart.transform_regression('PC0', self.y).mark_line(color='black').encode(
            color=alt.Color(legend=None),
        )
        parameters = chart.transform_regression('PC0', self.y, params=True).transform_calculate(
            intercept='datum.coef[0]',
            slope='datum.coef[1]',
        ).mark_text(align='left', color='black').encode(
            x=alt.value(300), 
            y=alt.value(20),
            text='slope:Q',
            tooltip=[alt.Tooltip('slope:Q', title='Slope'), alt.Tooltip('intercept:Q', title='Intercept')]
        )
        
        return self.get_scree_plot() | alt.layer(chart, regression, parameters).resolve_scale(
            color='independent'
        ).interactive()
        

In [5]:
data = pd.read_csv('tmp/developing_countries.csv')

# Calculate the percentage of unexplained happiness for each happiness score
data['Residual-to-happiness ratio'] = data['Dystopia residual'] / data['Happiness score']

# Analysis of the Effects of Urbanization on the Dystopia Residual

**Prediction of Dystopia Residual-to-Happiness Ratio Using Positive Indicators of Urbanization**

In [6]:
urbanization_residual_ratio = LinearRegressionModel(data.copy(), URBANIZATION_INDICATORS, 'Residual-to-happiness ratio')

C:\Users\rekak\AppData\Local\Temp\ipykernel_7960\3886339525.py:22: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_impute = df_thresh.fillna(df_thresh.mean())


In [7]:
urbanization_residual_ratio.display_properties()

R²: 0.24367351915284663
MSE: 0.0072329257315731765
Y-intercept: 0.5265862569149335
Coefficients:


,variable,coefficient
0,"Manufacturing, value added (% of GDP)",5.495278e-04
1,"Industry (including construction), value added...",-5.956130e-07
2,Employment in services (% of total employment)...,-1.318317e-03
3,Individuals using the Internet (% of population),-9.990781e-04
4,"Air transport, passengers carried",7.554100e-11
5,"Commercial bank branches (per 100,000 adults)",-2.456992e-04
6,Urban population (% of total),1.190491e-03
7,"Air transport, freight (million ton-km)",-1.091116e-06
8,Employment in industry (% of total employment)...,-3.494451e-03


In [8]:
urbanization_residual_ratio.visualize()

C:\Users\rekak\anaconda3\envs\test\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [9]:
urbanization_residual_ratio.get_predictions_table()

,"Manufacturing, value added (% of GDP)","Industry (including construction), value added per worker (constant 2010 US$)",Employment in services (% of total employment) (modeled ILO estimate),Individuals using the Internet (% of population),"Air transport, passengers carried","Commercial bank branches (per 100,000 adults)",Urban population (% of total),"Air transport, freight (million ton-km)",Employment in industry (% of total employment) (modeled ILO estimate),Predicted Residual-to-happiness ratio,Actual Residual-to-happiness ratio
54,12.666669,23036.702831,50.242001,42.805461,1.814317e+07,13.890841,55.169177,985.946746,21.684000,0.397612,0.429242
154,11.477828,9741.950131,25.670000,9.800000,1.814317e+07,2.509864,35.460000,985.946746,6.372000,0.503085,0.597738
400,4.631496,653.491899,45.643002,11.310000,4.751850e+05,5.581946,40.628000,8.349389,19.061001,0.437685,0.646503
67,13.621081,14666.319798,63.249001,56.656300,1.063498e+06,60.136678,73.990000,1.530950,29.895000,0.354271,0.213350
94,11.562774,46692.752294,67.337997,76.630000,1.500676e+07,16.328572,87.360000,1392.236000,23.284000,0.358036,0.401177
...,...,...,...,...,...,...,...,...,...,...,...
16,14.181585,25634.558099,76.170998,68.043064,1.424518e+07,13.168812,91.503000,243.772567,23.566999,0.374868,0.431396
284,22.805134,5798.523220,32.549999,25.073304,2.741388e+06,3.410647,30.082000,5.065286,16.350000,0.445745,0.342787
42,12.666669,23036.702831,67.765999,42.805461,1.814317e+07,13.890841,55.169177,985.946746,14.650000,0.399089,0.454835
451,7.552274,12562.920681,35.173000,27.852579,1.545730e+05,4.513118,42.976000,81.557610,10.636000,0.461867,0.404676


**Prediction of Dystopia Residual Values Using Positive Indicators of Urbanization**

In [10]:
urbanization_residual_value = LinearRegressionModel(data.copy(), URBANIZATION_INDICATORS, 'Dystopia residual')

C:\Users\rekak\AppData\Local\Temp\ipykernel_7960\3886339525.py:22: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_impute = df_thresh.fillna(df_thresh.mean())


In [11]:
urbanization_residual_value.display_properties()

R²: 0.08566029586768176
MSE: 0.40954043060408907
Y-intercept: 1.5895260345371953
Coefficients:


,variable,coefficient
0,"Manufacturing, value added (% of GDP)",1.251405e-02
1,"Industry (including construction), value added...",-3.592635e-06
2,Employment in services (% of total employment)...,3.343891e-03
3,Individuals using the Internet (% of population),-2.340448e-03
4,"Air transport, passengers carried",3.153496e-09
5,"Commercial bank branches (per 100,000 adults)",1.207791e-03
6,Urban population (% of total),8.672803e-03
7,"Air transport, freight (million ton-km)",-1.298663e-05
8,Employment in industry (% of total employment)...,-1.197982e-02


In [12]:
urbanization_residual_value.visualize()

C:\Users\rekak\anaconda3\envs\test\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [13]:
urbanization_residual_value.get_predictions_table()

,"Manufacturing, value added (% of GDP)","Industry (including construction), value added per worker (constant 2010 US$)",Employment in services (% of total employment) (modeled ILO estimate),Individuals using the Internet (% of population),"Air transport, passengers carried","Commercial bank branches (per 100,000 adults)",Urban population (% of total),"Air transport, freight (million ton-km)",Employment in industry (% of total employment) (modeled ILO estimate),Predicted Dystopia residual,Actual Dystopia residual
26,4.721174,48064.581209,49.279999,79.000000,2.331308e+06,13.890841,55.343000,751.143130,14.370000,1.778023,1.762482
109,11.404228,24233.030510,69.176003,66.028713,1.572605e+06,21.785458,77.735000,6.027344,18.610001,2.204388,3.351680
364,11.858271,28550.926871,71.476997,56.167394,2.082104e+07,10.427601,65.850000,833.930948,23.301001,2.102288,1.510909
82,12.666669,23036.702831,42.685001,42.805461,1.814317e+07,13.890841,55.169177,985.946746,26.881001,1.925454,1.322000
329,17.589085,32286.620697,58.112000,75.985366,7.376934e+06,29.290157,60.105000,221.814000,31.667000,1.907796,1.797723
...,...,...,...,...,...,...,...,...,...,...,...
235,12.666669,23036.702831,53.432999,42.805461,1.814317e+07,13.890841,55.169177,985.946746,30.452999,1.918603,1.677000
31,12.666669,23036.702831,63.708000,42.805461,1.814317e+07,13.890841,55.169177,985.946746,35.255001,1.895434,1.743000
69,14.681248,15267.848602,63.087002,63.410101,1.110884e+06,50.868308,74.669000,2.537000,29.899000,2.135258,0.996139
365,12.666669,23036.702831,71.599998,42.805461,1.814317e+07,13.890841,55.169177,985.946746,23.243000,2.065726,1.369000


# Analysis of the Effects of Negative Indicators of Urbanization on the Dystopia Residual

**Prediction of Dystopia Residual-to-Happiness Ratio Using Negative Indicators of Urbanization**

In [14]:
neg_urbanization_residual_ratio = LinearRegressionModel(data.copy(), ANTI_URBANIZATION_INDICATORS, 'Residual-to-happiness ratio')

C:\Users\rekak\AppData\Local\Temp\ipykernel_7960\3886339525.py:22: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_impute = df_thresh.fillna(df_thresh.mean())


In [15]:
neg_urbanization_residual_ratio.display_properties()

R²: 0.2079760625467617
MSE: 0.010594316799007608
Y-intercept: 0.3349246221350098
Coefficients:


,variable,coefficient
0,"Agriculture, forestry, and fishing, value adde...",3.824377e-08
1,Employment in agriculture (% of total employme...,2.343843e-03


In [16]:
neg_urbanization_residual_ratio.visualize()

C:\Users\rekak\anaconda3\envs\test\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [17]:
neg_urbanization_residual_ratio.get_predictions_table()

,"Agriculture, forestry, and fishing, value added per worker (constant 2010 US$)",Employment in agriculture (% of total employment) (modeled ILO estimate),Predicted Residual-to-happiness ratio,Actual Residual-to-happiness ratio
215,1520.613156,26.646000,0.397437,0.307099
196,20761.228907,15.008000,0.370895,0.306908
82,20761.228907,30.434999,0.407053,0.298218
172,20761.228907,43.860001,0.438520,0.342005
30,10237.083206,1.047000,0.337770,0.272080
...,...,...,...,...
419,563.038115,71.385002,0.502261,0.404964
140,20761.228907,37.605999,0.423861,0.360235
240,463.495620,72.286003,0.504369,0.654219
437,20761.228907,7.292000,0.352810,0.468621


**Prediction of Dystopia Residual Values Using Positive Indicators of Urbanization**

In [18]:
neg_urbanization_residual_value = LinearRegressionModel(data.copy(), ANTI_URBANIZATION_INDICATORS, 'Dystopia residual')

C:\Users\rekak\AppData\Local\Temp\ipykernel_7960\3886339525.py:22: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_impute = df_thresh.fillna(df_thresh.mean())


In [19]:
neg_urbanization_residual_value.display_properties()

R²: 0.014846042755988553
MSE: 0.27466554779428953
Y-intercept: 2.0862941992432478
Coefficients:


,variable,coefficient
0,"Agriculture, forestry, and fishing, value adde...",3.138452e-07
1,Employment in agriculture (% of total employme...,-2.012457e-03


In [20]:
neg_urbanization_residual_value.visualize()

C:\Users\rekak\anaconda3\envs\test\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [21]:
neg_urbanization_residual_value.get_predictions_table()

,"Agriculture, forestry, and fishing, value added per worker (constant 2010 US$)",Employment in agriculture (% of total employment) (modeled ILO estimate),Predicted Dystopia residual,Actual Dystopia residual
269,4538.154879,30.360001,2.026620,1.535860
130,2610.510124,18.679001,2.049523,3.221340
352,2293.323952,32.962002,2.020679,1.978610
128,20761.228907,24.868000,2.042764,1.445000
6,5736.635217,38.203999,2.011211,1.490442
...,...,...,...,...
453,294.738272,67.121002,1.951309,2.441910
208,20761.228907,17.082001,2.058433,1.904000
39,20761.228907,10.589000,2.071500,1.684000
210,11285.807363,2.309000,2.085189,2.280850
